In [97]:
import pandas as pd
import numpy as np
import torch
import os
# os.chdir('/home/jiageng/Documents/fhr/pipeline/')
os.chdir('/home/jiageng/Documents/fhr/pygcn/pygcn')
import snf

In [98]:
def stdNormalize(df):
    std = df.std().fillna(1)
    mean = df - df.mean()
    df_norm = mean / std
    return np.array(df_norm)

In [99]:
def rowNormalize(mx):
    """Row-normalize matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = np.diag(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [100]:
from scipy.sparse import coo_matrix

def sparse_tensor_from_arr(arr):
    """Convert numpy array to torch sparse tensor"""
    # Convert numpy array to scipy sparse matrix
    sparse_sp = coo_matrix(arr)
    
    # Convert scipy sparse matrix to torch sparse tensor
    sparse_tensor = torch.sparse_coo_tensor(
        torch.LongTensor([sparse_sp.row, sparse_sp.col]),
        torch.FloatTensor(sparse_sp.data),
        torch.Size(sparse_sp.shape)
    )
    
    return sparse_tensor

Prepare labels

In [101]:
df_labels = pd.read_csv('/home/jiageng/Documents/fhr/annotations/fhr-annotations.tsv',sep='\t').set_index('PUBLIC_ID').query('risk != -1')
df_labels['risk'] += 1

Prepare features

In [102]:
# subset to samples with fhr labels
data = pd.read_csv('/home/jiageng/Documents/fhr/matrices/gene_exp_matrix_k500.tsv',sep='\t').set_index('PUBLIC_ID')
if 'SAMPLE' in data.columns:
    data = data.drop(columns=['SAMPLE'])
print(data.shape)

(806, 500)


Use all samples with features, handle the missing labels

In [103]:
# use all rna seq samples
public_ids = data.index
print(len(public_ids))

806


In [104]:
feature_norm_mtd = '' 

if feature_norm_mtd == 'stdnorm':
    features = torch.tensor(stdNormalize(data.loc[public_ids]),dtype=torch.float32)
elif feature_norm_mtd == 'rownorm':
    features = torch.tensor(rowNormalize(data.loc[public_ids].values),dtype=torch.float32)
else:
    # default is to use raw log tpm+1 values
    features = torch.tensor(data.loc[public_ids].values,dtype=torch.float32)
print(features.size())

torch.Size([806, 500])


In [105]:
risk = df_labels.reindex(pd.Index(public_ids)).loc[public_ids]['risk'].fillna(-1).astype(int)
labels = torch.tensor(risk.values,dtype=torch.long)

In [106]:
# missing values are set to -1
risk.value_counts()

risk
 1    403
 2    216
-1    105
 3     82
Name: count, dtype: int64

In [107]:
idx_labeled = np.where(risk != -1)[0]
idx_unlabeled = np.where(risk == -1)[0]
idx_val = torch.tensor(idx_labeled[::3])
idx_test = idx_val
idx_train = torch.tensor([idx for idx in idx_labeled if idx not in idx_val])
print('Un-labeled',len(idx_unlabeled))
print('Labeled train',len(idx_train))
print('Labeled val',len(idx_val))

Un-labeled 105
Labeled train 467
Labeled val 234


In [108]:
# 1 = SR, 2 = GHR, 3 = FHR
risk.iloc[idx_train].value_counts()

risk
1    268
2    149
3     50
Name: count, dtype: int64

In [109]:
# 1 = SR, 2 = GHR, 3 = FHR
# idx_val and idx_test are the same for now
risk.iloc[idx_val].value_counts()

risk
1    135
2     67
3     32
Name: count, dtype: int64

Prepare adjacency matrix

In [110]:
import snf
aff = snf.make_affinity(stdNormalize(data), metric='cosine', K=100, mu=0.5)
print(aff.shape)

/home/jiageng/Documents/fhr/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jiageng/Documents/fhr/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(806, 806)


Increase the sparsity of the matrix by 0-clipping small values

The sparser the matrix, the higher accuracy of the GCN

In [111]:
# Method 1 - sparsen before row-normalization
pctile = 0.95
threshold = np.quantile(aff,pctile)
print(pctile, threshold)
aff_clip = aff.copy()
aff_clip[aff_clip < threshold] = 0
adj = sparse_tensor_from_arr(rowNormalize(aff_clip))

0.95 0.18962869481000272


In [136]:
# Method 2 - sparsen after row-normalization
# this is the method that works for mutations
adj = rowNormalize(aff)
pctile=0.95
threshold = np.quantile(adj,pctile)
print(pctile, threshold)
adj[adj < threshold] = 0
adj = sparse_tensor_from_arr(adj)

0.95 0.003963189074209203


# Train Model

In [144]:
import time
import argparse
import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy, balanced_accuracy
from pygcn.models import GCN
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args = parser.parse_args(['--epochs=200','--dropout=0.9','--hidden=16','--weight_decay=5e-4'])
args.cuda = not args.no_cuda and torch.cuda.is_available()
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

def balanced_accuracy(output, labels):
    # output, labels - torch tensors
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    
    # Calculate the frequency of each label
    label_counts = torch.bincount(labels)
    weights = 1.0 / label_counts.float()
    
    # Apply weights to the correct predictions
    weighted_correct = correct * weights[labels]
    weighted_accuracy = weighted_correct.sum() / weights[labels].sum()
    
    return weighted_accuracy

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = balanced_accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = balanced_accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = balanced_accuracy(output[idx_test], labels[idx_test])
    acc_test_unbal = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
          "unweighted accuracy= {:.4f}".format(acc_test_unbal.item()))

# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 2.3515 acc_train: 0.2637 loss_val: 1.1749 acc_val: 0.3387 time: 0.0238s
Epoch: 0002 loss_train: 1.3149 acc_train: 0.3707 loss_val: 1.0689 acc_val: 0.5378 time: 0.0243s
Epoch: 0003 loss_train: 1.1371 acc_train: 0.4499 loss_val: 1.1364 acc_val: 0.4386 time: 0.0175s
Epoch: 0004 loss_train: 1.5280 acc_train: 0.3279 loss_val: 1.1369 acc_val: 0.4332 time: 0.0150s
Epoch: 0005 loss_train: 1.0800 acc_train: 0.4915 loss_val: 1.1072 acc_val: 0.5046 time: 0.0149s
Epoch: 0006 loss_train: 1.2009 acc_train: 0.3709 loss_val: 1.0683 acc_val: 0.5586 time: 0.0180s
Epoch: 0007 loss_train: 1.0670 acc_train: 0.4779 loss_val: 1.0671 acc_val: 0.4912 time: 0.0153s
Epoch: 0008 loss_train: 1.0947 acc_train: 0.4175 loss_val: 1.0671 acc_val: 0.4837 time: 0.0139s
Epoch: 0009 loss_train: 1.1208 acc_train: 0.4296 loss_val: 1.0573 acc_val: 0.4788 time: 0.0134s
Epoch: 0010 loss_train: 1.1741 acc_train: 0.4143 loss_val: 1.0366 acc_val: 0.5111 time: 0.0122s
Epoch: 0011 loss_train: 1.1149 acc_train

In [138]:
# FFNN control - only consider self values
adj = torch.eye(adj.size(0)).to_sparse()

In [145]:
# Method 2 - sparsen after row-normalization
# this is the method that works for mutations
adj = rowNormalize(aff)
pctile=0.
threshold = np.quantile(adj,pctile)
print(pctile, threshold)
adj[adj < threshold] = 0
adj = sparse_tensor_from_arr(adj)

0.0 7.167838629011553e-07
